# 크롤링

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import urllib.request 
from selenium.webdriver import Chrome
import re     
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt
import pandas as pd

In [2]:
def get_videos (channel): # 채널 내 영상들의 제목/URL 크롤링
    
    # 브라우저 열기
    delay=3
    browser = Chrome()
    browser.implicitly_wait(delay)

    # url을 받아 창을 열고, 최대화
    start_url  = channel
    browser.get(start_url)  
    browser.maximize_window()

    # 영상목록 클릭 후 body 찾기
    body = browser.find_element_by_css_selector('body')

    
    # 5번 밑으로 내리기 -> 타임라인 업데이트를 위해서!
    num_of_pagedowns = 2
    while num_of_pagedowns:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        num_of_pagedowns -= 1
    
    # 파싱 후 크롤링
    html0 = browser.page_source
    soup = BeautifulSoup(html0,'lxml')
    
    videos = soup.select('h3 > a')

    title = []
    url = []

    for video in videos:
        if video.get('href')[:7] != '/watch?':
            pass
        else:
            title.append(video.text)
            url.append('https://www.youtube.com/' + video.get('href'))
    
   

    browser.close() 
    return url, title


'https://www.youtube.com/channel/UC3HH4dZj3CZmrCC0MuDliww'

'https://www.youtube.com/channel/UC3HH4dZj3CZmrCC0MuDliww'

In [3]:
url_list, title_list = get_videos('https://www.youtube.com/channel/UC3HH4dZj3CZmrCC0MuDliww')

In [4]:
def get_info(title_list, url_list,cnt):

    df = pd.DataFrame({
        '날짜':[],
        '제목':[],
        '조회수':[],
        '댓글 수':[],
        'URL':[],
        '좋아요 수':[],
        '싫어요 수':[],
        '댓글' : []})

    for i in reversed(range(cnt)): # 최신 순으로 추출하기 위해
        name = title_list[i]
        start_url = url_list[i]
        
        browser = Chrome()
        browser.get(start_url)
        browser.maximize_window()
        body = browser.find_element_by_tag_name('body')
        
        num_page_down = 5 # 20번 이상이 적당함.
        while num_page_down:
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(1.5)
            num_page_down -= 1
            
            
        html0 = browser.page_source
        html = BeautifulSoup(html0,'html.parser')

        start_date = html.find('div',
                               {'id':'info-strings','class':'style-scope ytd-video-primary-info-renderer'}).text
        start_date = re.sub(r'.', '', start_date, count = 1)
        
        view = html.find('span',
                         {'class':'view-count style-scope ytd-video-view-count-renderer'}).text
        view = re.sub(r'.', '', view, count = 4)
        
        time.sleep(1)
        
        # 댓글 수
        try:
            comment_num = html.find('yt-formatted-string',
                                {'class' : "count-text style-scope ytd-comments-header-renderer"}).text
        except:
            pass

        #좋아요수
        likes_num = html.find('yt-formatted-string',{'id':'text',
                                                     'class':'style-scope ytd-toggle-button-renderer style-text',
                                                     'aria-label':re.compile('좋아요')}).text+'개'


        #싫어요수
        unlikes_num = html.find('yt-formatted-string',{'id':'text',
                                                       'class':'style-scope ytd-toggle-button-renderer style-text',
                                                       'aria-label':re.compile('싫어요')}).text+'개'
        
        # 댓글
        comment = []
        comment_data = html.find_all('ytd-comment-renderer',
                           {'class':'style-scope ytd-comment-thread-renderer'})
        
        for i in range(len(comment_data)):
            comments = comment_data[i].find('yt-formatted-string',
                                       {'id':'content-text','class':'style-scope ytd-comment-renderer'}).text

            try:
                tmp1 = comment_data[i].find('span',{'id':'vote-count-left'}).text
                like = "".join(re.findall('[0-9]',tmp1)) + "개"
            except:
                like = 0

            tmp2 = comment_data[i].find('a',{'id':'author-text'}).find('span').text
            youtube_id = "".join(re.findall('[가-힣0-9a-zA-Z]',tmp2))

            comment.append([youtube_id, comments, like])
            
            
        insert_data = pd.DataFrame({
        '날짜':[start_date],
        '제목':[name],
        '조회수':[view],
        '댓글 수':[comment_num],
        'URL':[start_url],
        '좋아요 수':[likes_num],
        '싫어요 수':[unlikes_num],
        '댓글' : [comment]})

        df = df.append(insert_data)
        browser.quit()
        
    return df


In [5]:
df = get_info(title_list, url_list,2)
df

,날짜,제목,조회수,댓글 수,URL,좋아요 수,싫어요 수,댓글
0,021. 8. 4.,십수년전부터 외계인의 공격을 미리 대비한 남자 1부,"29,789회",댓글 73개,https://www.youtube.com//watch?v=Dazy1fBg1xA,523개,7개,"[[엄마는이계인, 늘 잘 보고 있습니다 영일남님이 노인을 위한 나라는 없다 영화도 ..."
0,021. 8. 4.,십수년전부터 외계인의 공격을 미리 대비한 남자 2부,"24,897회",댓글 101개,https://www.youtube.com//watch?v=L7hxMD8rYrw,418개,7개,"[[금금, 영일남님 나레이션 너무 평온하고 좋은 것... 클로버필드 3편도 리뷰 해..."


In [4]:
def get_video(): # 검색어를 입력해 영상들의 기본 정보를 크롤링
    feature = input('검색어를 입력하시오 : ')
    
    driver = Chrome()
    driver.get('https://www.youtube.com')
    
    n = 3
    while n > 0:
        print('웹페이지를 불러오는 중입니다..' + '..' * n)
        time.sleep(1)
        n -= 1
        
    src = driver.find_element_by_xpath('//*[@id="search"]')
    src.send_keys(feature)
    src.send_keys(Keys.RETURN)
    
    n = 2
    while n > 0:
        print('검색 결과를 불러오는 중입니다..' + '..' * n)
        time.sleep(1)
        n -= 1
        
    print('데이터 수집 중입니다....')
               
    html = driver.page_source
    soup = BeautifulSoup(html)
    
    df_title = []
    df_link = []
    df_writer = []
    df_view = []
    df_date = []
    
    for i in range(len(soup.find_all('ytd-video-meta-block', 'style-scope ytd-video-renderer byline-separated'))):
        title = soup.find_all('a', {'id' : 'video-title'})[i].text.replace('\n', '')
        link = 'https://www.youtube.com/' + soup.find_all('a', {'id' : 'video-title'})[i]['href']
        writer = soup.find_all('ytd-channel-name', 'long-byline style-scope ytd-video-renderer')[i].text.replace('\n', '').split(' ')[0]
        view = soup.find_all('ytd-video-meta-block', 'style-scope ytd-video-renderer byline-separated')[i].text.split('•')[1].split('\n')[3]
        date = soup.find_all('ytd-video-meta-block', 'style-scope ytd-video-renderer byline-separated')[i].text.split('•')[1].split('\n')[4]
    
        df_title.append(title)
        df_link.append(link)
        df_writer.append(writer)
        df_view.append(view)
        df_date.append(date)
          
    df_just_video = pd.DataFrame(columns=['영상제목', '채널명', '영상url', '조회수', '영상등록날짜'])

    df_just_video['영상제목'] = df_title
    df_just_video['채널명'] = df_writer
    df_just_video['영상url'] = df_link
    df_just_video['조회수'] = df_view
    df_just_video['영상등록날짜'] = df_date
          
    df_just_video.to_csv('df_just_video.csv', encoding='utf-8-sig', index=False)
    
    driver.close()
          
    result = input('데이터프레임 저장이 완료되었습니다! 데이터프레임을 조회하시겠습니까? (y/n)')
    if result == 'y':
        display(df_just_video)
        question = input('원하는 영상을 재생하시겠습니까? (y/n)')
        if question == 'y':
            button = int(input('재생하고자 하는 영상의 번호(출력된 표 가장 왼쪽의 번호)를 입력해주세요.'))
            driver = Chrome(path)
            driver.get(df_just_video['영상url'][button])
        else:
            return '프로그램을 종료합니다.'
    else:
        return '프로그램을 종료합니다.'

In [5]:
get_video()

검색어를 입력하시오 : 영화
웹페이지를 불러오는 중입니다........
웹페이지를 불러오는 중입니다......
웹페이지를 불러오는 중입니다....
검색 결과를 불러오는 중입니다......
검색 결과를 불러오는 중입니다....
데이터 수집 중입니다....


c:\users\samsung\appdata\local\programs\python\python37\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file c:\users\samsung\appdata\local\programs\python\python37\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


데이터프레임 저장이 완료되었습니다! 데이터프레임을 조회하시겠습니까? (y/n)y


,영상제목,채널명,영상url,조회수,영상등록날짜
0,러시아에서 만든 레전드 1위 '다크 판타지' 액션!! [영화리뷰 결말포함],동네비디오방,https://www.youtube.com//watch?v=Vf9Nnyux9dc,조회수 1.3만회,9시간 전
1,[영화리뷰/결말포함] 조폭 조직에서 ACE였던 남자가 교도소에 버려지면 벌어지는 충...,영반,https://www.youtube.com//watch?v=CBCmPB3bW5A,조회수 1.6천회,2시간 전
2,[영화리뷰/결말포함] 힘숨찐을 이기면 150만원을 준다는 일진들 일진 시즌3 대가리,JKwondo,https://www.youtube.com//watch?v=19HvILzbI1c,조회수 1.7천회,2시간 전
3,탈출불가의 소말리아에서 한국인의 기발함으로 탈출한 실화기반 화제의 신작영화 [모가디슈],고몽,https://www.youtube.com//watch?v=4IYpOzxpHaI,조회수 78만회,3주 전
4,친구의 엄마 덕분에 그 맛을 알아버린 죽마고우 [영화리뷰/결말포함],달타냥씨네마,https://www.youtube.com//watch?v=zdwq8oRlF_E,조회수 186만회,2개월 전
5,너무 충격적 실화였기에 개봉자체를 중국에서 끝까지 막고자 했던 영화,제이나나,https://www.youtube.com//watch?v=R-bsMpQMX7M,조회수 2.9천회,1시간 전
6,명품 브랜드 ‘구찌🛍️’ 를 개막장 영화로 뽑았어.. 그런데 자레드 레토가..ㄷㄷ,기묘한,https://www.youtube.com//watch?v=GmUZHuDrfSk,조회수 3.1만회,1일 전
7,자막) 전 세계에서 흥행 수익 3천만 달러를 기록한 올해의 공포 영화 (영화리뷰 결...,영화대장,https://www.youtube.com//watch?v=wMHj1_nEWEg,조회수 7.6만회,4일 전
8,진짜 야하고 잔인한.. 여자 교도소에서 벌어지는 충격적인 일들 [영화리뷰 결말포함 ...,무비가가,https://www.youtube.com//watch?v=ntHCNs_X6qc,조회수 2.2만회,스트리밍 시간: 21시간 전
9,"[거의잡스럽다] 영화 ""괴물"" - 봉준호 감독, 송강호 주연",엠장기획,https://www.youtube.com//watch?v=jB0-DJfz7_I,조회수 30만회,4일 전


원하는 영상을 재생하시겠습니까? (y/n)n


'프로그램을 종료합니다.'